# Hopfield Network

In [11]:
!pip install scikit-learn

   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ------- -------------------------------- 1.6/8.7 MB 12.0 MB/s eta 0:00:01
   ---------------------------------------  8.7/8.7 MB 26.9 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 23.6 MB/s  0:00:00
   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   --------- ------------------------------ 9.4/38.5 MB 49.1 MB/s eta 0:00:01
   --------------- ------------------------ 15.2/38.5 MB 43.6 MB/s eta 0:00:01
   ---------------------- ----------------- 22.0/38.5 MB 36.7 MB/s eta 0:00:01
   -------------------------------- ------- 31.2/38.5 MB 38.8 MB/s eta 0:00:01
   ---------------------------------------  38.3/38.5 MB 38.0 MB/s eta 0:00:01
   ---------------------------------------- 38.5/38.5 MB 34.0 MB/s  0:00:01

   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [s

In [14]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
import zipfile

In [5]:
!curl https://frasca.di.unimi.it/MLDNN/input_data.zip -O data.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
  5 4581k    5  242k    0     0  60000      0  0:01:18  0:00:04  0:01:14 60018
100 4581k  100 4581k    0     0  1010k      0  0:00:04  0:00:04 --:--:-- 1037k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: data.zip


In [7]:
with zipfile.ZipFile("input_data.zip", 'r') as zip_ref:
     zip_ref.extractall("unzipped_data")
import pickle as pk
import numpy as np

with open("unzipped_data/input_data.pkl", "rb") as f:
     data_dict = pk.load(f)

W = data_dict['similarity_matrix']
labels = data_dict['labels']
true_labels = data_dict['true_labels']

In [8]:
import numpy as np
# Hopfield network as local optimizer
class HopfieldNetworkLO:
    def __init__(self, weight_matrix):
        """Initialize network with a given weight matrix."""
        self.W = np.array(weight_matrix)  # Convert weight matrix to numpy array
        self.N = self.W.shape[0]  # Number of neurons

    def run_dynamics(self, labels, max_iterations=100):
        """Perform asynchronous updates for neurons labeled as '0'."""
        state = np.array(labels, dtype=float)  # Convert labels to float array
        free_neurons = np.where(state == 0)[0]  # Indices of neurons to update
        fixed_neurons = np.where(state != 0)[0]  # Indices of fixed neurons

        # Compute external influence from fixed neurons
        b = np.dot(self.W[:, fixed_neurons], state[fixed_neurons])  # Activation threshold

        for _ in range(max_iterations):
            updated = False
            for i in free_neurons:
                # Compute total input sum (including fixed neurons' influence)
                activation = np.dot(self.W[i], state) - b[i]
                new_state = 1 if activation > 0 else -1

                if new_state != state[i]:
                    state[i] = new_state
                    updated = True

            if not updated:  # Stop if no changes occur
                break

        return state



In [9]:
outcome = HopfieldNetworkLO(W).run_dynamics(labels)



In [17]:
# Calcolo dell'accuratezza della Hopfield Network
accuracy = np.mean(outcome == true_labels)
print(f"Accuratezza: {accuracy * 100:.2f}%")
score = f1_score(outcome, true_labels)
print(f'F1 score = {score}')

Accuratezza: 93.30%
F1 score = 0.9189842805320435
